In [ ]:
# This allows changes in chakra.py to be automatically re-imported
# (this is tricky with OOP though, to be used with care)
%load_ext autoreload
%autoreload 1
%aimport chakra

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Get the default params
from oggm import cfg
cfg.initialize(logging_level='WORKFLOW')
cfg.PARAMS['cfl_number'] = 0.01  # less numerical instabilities
cfg.PARAMS['use_multiprocessing'] = False

## Velocity-forced viscoplastic calving

First, we'll define some constants and a preliminary function for yield strength.  Then, we'll write a function that takes profiles and velocity from any model and outputs a viscoplastic frontal ablation rate.

In [ ]:
## Global constants
G = 9.8 # acceleration due to gravity in m/s^2
RHO_ICE = 920.0 #ice density kg/m^3
RHO_SEA = 1020.0 #seawater density kg/m^3

def tau_y(tau0=150e3, yield_type='constant', bed_elev=None, thick=None, mu=0.01):
    """Functional form of yield strength.
    Can do constant or Mohr-Coulomb yield strength.  Ideally, the glacier's yield type
    ('constant' or 'variable') would be saved in a model instance.

    Parameters
    ----------
    tau0: float, optional
        Initial guess for yield strength [Pa]. Default is 150 kPa.
    yield_type: str, optional
        'constant' or 'variable' (Mohr-Coulomb) yielding. Default is constant.
    bed_elev: float, optional
        Bed elevation, dimensional [m]. The default is None.
    thick: float, optional
        Ice thickness, dimensional [m]. The default is None.
    mu: float, optional
        Mohr-Coulomb cohesion, a coefficient between 0 and 1. Default is 0.01.

    Returns
    -------
    tau_y: float
        The yield strength for these conditions.
    """
    if yield_type=='variable':
        try:
            if bed_elev<0:
                D = -1*bed_elev #Water depth D the nondim bed topography value when Z<0
            else:
                D = 0
        except:
            print('You must set a bed elevation and ice thickness to use variable yield strength.')
        N = RHO_ICE*G*thick - RHO_SEA*G*D #Normal stress at bed
        ty = tau0 + mu*N
    else: #assume constant if not set
        ty = tau0
    return ty


def balance_thickness(yield_strength, bed_elev):
    """Ice thickness such that the stress matches the yield strength.
    
    Parameters
    ----------
    yield_strength: float
        The yield strength near the terminus.  
        If yield type is constant, this will of course be the same everywhere.  If yield type is 
        variable (Mohr-Coulomb), the yield strength at the terminus could differ from elsewhere.
    bed_elev: float
        Elevation of glacier bed at the terminus
        
    Returns
    -------
    Hy: float
        The ice thickness for stress balance at the terminus.
    """
    
    if bed_elev<0:
        D = -1*bed_elev
    else:
        D = 0
    return (2*yield_strength/(RHO_ICE*G)) + np.sqrt((RHO_SEA*(D**2)/RHO_ICE)+(2*yield_strength/(RHO_ICE*G)))
## TODO: Check on exponent on last term.  In Ultee & Bassis 2016, this is squared, but in Ultee & Bassis 2020 supplement, it isn't.
    

In [ ]:
def fa_from_velocity(profile, model_velocity, terminus_mb=None, verbose=False,
                    tau0=150e3, yield_type='constant', mu=0.01,
                    trim_profile=0):
    """Compute frontal ablation given velocity forcing
    
    Parameters
    ----------
    profile: NDarray
        The current profile (x, surface, bed) as calculated by the base model
        Unlike core SERMeQ, these should be DIMENSIONAL [m].
    model_velocity: array
        Velocity along the flowline [m/a] as calculated by the base model 
        Should have values for the points nearest the terminus...otherwise doesn't matter if this
        is the same shape as the profile array.
    terminus_mb : float, optional
        Mass balance nearest the terminus [m/a]. Default None...TODO: set default behavior
    verbose: Boolean, optional
        Whether to print component parts for inspection.  Default False.
        
    tau0: float, optional
        This glacier's yield strength [Pa]. Default is 150 kPa.
    yield_type: str, optional
        'constant' or 'variable' (Mohr-Coulomb) yielding. Default is constant.
    mu: float, optional
        Mohr-Coulomb cohesion, a coefficient between 0 and 1. Default is 0.01.
        Only used if we have variable yield
        
    trim_profile: int, optional
        How many grid cells at the end of the profile to ignore.  Default is 1.
        If the initial profile is set by k-calving (as in testing) there can be a 
        weird cliff shape with very thin final grid point and large velocity gradient
    
    Returns
    -------
    fa_viscoplastic: float
        Frontal ablation rate [m/a] based on viscoplastic assumptions
    """
    last_index=-1*(trim_profile+1) ## remove lowest cells if needed
        
    ## Ice thickness and yield thickness nearest the terminus
    se_terminus = profile[1][last_index]
    bed_terminus = profile[2][last_index]
    H_terminus = se_terminus - bed_terminus
    tau_y_terminus = tau_y(tau0=tau0, bed_elev=bed_terminus, thick=H_terminus)
    Hy_terminus = balance_thickness(yield_strength=tau_y_terminus, bed_elev=bed_terminus)
    U_terminus = model_velocity[last_index] ## velocity, assuming last point is terminus
    
    ## Ice thickness and yield thickness at adjacent point
    se_adj = profile[1][last_index-1]
    bed_adj = profile[1][last_index-1]
    H_adj = se_adj - bed_adj
    tau_y_adj = tau_y(tau0=tau0,bed_elev=bed_adj, thick=H_adj)
    Hy_adj = balance_thickness(yield_strength=tau_y_adj, bed_elev=bed_adj)
    U_adj = model_velocity[last_index-1]
    
    
    # Gradients
    dx_term = abs(profile[0][last_index-1] - profile[0][last_index]) ## check grid spacing close to terminus
    dHdx = (H_adj-H_terminus)/dx_term
    dHydx = (Hy_adj-Hy_terminus)/dx_term
    dUdx = (U_adj - U_terminus)/ dx_term ## velocity gradient
    
    
    ## Group the terms
    dLdt_numerator = terminus_mb - (H_terminus * dUdx) - (U_terminus * dHdx)
    dLdt_denominator = dHydx - dHdx ## TODO: compute dHydx
    dLdt_viscoplastic = dLdt_numerator / dLdt_denominator
    
    fa_viscoplastic = U_terminus - dLdt_viscoplastic ## frontal ablation rate
    
    if verbose:
        print('For inspection on debugging - all should be DIMENSIONAL (m/a):')
#         print('profile_length={}'.format(profile_length))
        print('se_terminus={}'.format(se_terminus))
        print('bed_terminus={}'.format(bed_terminus))
        print('Hy_terminus={}'.format(Hy_terminus))
        print('U_terminus={}'.format(U_terminus))
        print('dx_term={}'.format(dx_term))
        print('Checking dLdt: terminus_mb = {}. \n H dUdx = {}. \n U dHdx = {}.'.format(terminus_mb, dUdx*H_terminus, U_terminus*dHdx)) 
        print('Denom: dHydx = {} \n dHdx = {}'.format(dHydx, dHdx))
        print('Viscoplastic dLdt={}'.format(dLdt_viscoplastic))
    else:
        pass
    
    return fa_viscoplastic


### Testing on an idealized profile

In [ ]:
from oggm.core.massbalance import ScalarMassBalance
no_mb = ScalarMassBalance()

#### BU bed

In [ ]:
bu_fl = chakra.bu_tidewater_bed()[0]

xc = bu_fl.dis_on_line * bu_fl.dx_meter / 1000
f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(xc, bu_fl.bed_h, color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

#### Put a k-calving profile on top of this and check what viscoplastic calving rates we'd get

In [ ]:
calving_k = 0.2 # set a relatively low calving

to_plot = None
keys = []
for flux_gate in [0.06, 0.10, 0.16]:
    
    model = chakra.KCalvingModel(chakra.bu_tidewater_bed(), mb_model=no_mb, 
                               flux_gate=flux_gate,
                               calving_use_limiter=True,
                               do_kcalving=True,
                                 calving_k=calving_k,
#                                apply_parameterization=k_calving,
                               )
    
    # long enough to reach approx. equilibrium 
    _, ds = model.run_until_and_store(2000)
    df_diag = model.get_diagnostics()
    
    if to_plot is None:
        to_plot = df_diag
    
    key = 'Flux gate={:.02f}. Calving rate: {:.0f} m yr-1'.format(flux_gate, model.calving_rate_myr)
    to_plot[key] = df_diag['surface_h']
    keys.append(key)
    
    # Plot of volume
    (ds.volume_m3 * 1e-9).plot(label=key);
    
plt.legend(); plt.ylabel('Volume [km$^{3}$]');
to_plot.index = xc

In [ ]:
cfg.PARAMS['calving_k']

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 5))
to_plot[keys].plot(ax=ax);
to_plot.bed_h.plot(ax=ax, color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

Not sure why these all claim a 0 calving rate...anyway, let's use their output for now to test some viscoplastic calving rates.  These are stored in a weird way, so there's only one ice velocity available, maybe the last one simulated?  Anyway...

In [ ]:
to_plot

In [ ]:
surface_profile = to_plot['Flux gate=0.16. Calving rate: 0 m yr-1']
bed_profile = to_plot['bed_h']
x = to_plot.index
model_U = to_plot['ice_velocity'] * cfg.SEC_IN_YEAR ## convert ice velocity to m/a

In [ ]:
max(model_U)

Okay, the maximum ice velocity is 10 m/a.  This is really slow.  Let's see how this goes.

In [ ]:
## Find index of the terminus
term_index = int(np.argwhere(surface_profile.values>0)[-1])

input_profile = (x.values[:term_index+1],
                 surface_profile.values[:term_index+1],
                 bed_profile.values[:term_index+1])
input_velocity = model_U.values[:term_index+1]

In [ ]:
no_mb.get_annual_mb(heights=surface_profile.values[0])

In [ ]:
testval = fa_from_velocity(profile=input_profile, model_velocity=input_velocity, terminus_mb=0, verbose=False,
                    tau0=150e3, yield_type='constant', mu=0.01)

In [ ]:
testval

Okay, we got a value that seems reasonable!  It's a low ablation rate but we'll take it.  Let's explore a bit.

In [ ]:
ty_tests = np.linspace(25e3, 500e3, num=50)
fa_results = [fa_from_velocity(profile=input_profile, model_velocity=input_velocity, terminus_mb=0, verbose=False,
                    tau0=ty, yield_type='constant', mu=0.01,trim_profile=1) for ty in ty_tests]

In [ ]:
fig, ax = plt.subplots()
ax.plot(1e-3*ty_tests, fa_results)
ax.set(xlabel='Yield strength [kPa]',
       ylabel='Frontal ablation rate')

In [ ]:
for ty in ty_tests[0:3]:
    t = fa_from_velocity(profile=input_profile, model_velocity=input_velocity, terminus_mb=0, verbose=True,
                    tau0=ty, yield_type='constant', mu=0.01, trim_profile=1)

In [ ]:
mb_tests = np.linspace(-10, 10, num=50)
fa_results_mb = [fa_from_velocity(profile=input_profile, model_velocity=input_velocity, terminus_mb=mb, verbose=False,
                    tau0=150e3, yield_type='constant', mu=0.01, trim_profile=1) for mb in mb_tests]

In [ ]:
fig, ax = plt.subplots()
ax.plot(mb_tests, fa_results_mb)
ax.set(xlabel='Terminus mass balance [m/a]',
       ylabel='Frontal ablation rate')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharey=True)
ax1.plot(1e-3*ty_tests, fa_results)
ax1.set(xlabel='Yield strength [kPa], mb=0',
       ylabel='Frontal ablation rate')
ax2.plot(mb_tests, fa_results_mb)
ax2.set(xlabel=r'Terminus mass balance [m/a], $\tau_y$=150 KPa')

We know our glacier is moving veeery slowly.  What does the frontal ablation rate look like if we scale up the velocity?

In [ ]:
v_scalings = np.linspace(0.1,20,num=50)
fa_results_v = [fa_from_velocity(profile=input_profile, model_velocity=v*np.asarray(input_velocity), terminus_mb=0, verbose=False,
                    tau0=150e3, yield_type='constant', mu=0.01,trim_profile=1) for v in v_scalings]

In [ ]:
fig, (ax1, ax2,ax3) = plt.subplots(1,3,sharey=True, figsize=(10,4))
ax1.plot(1e-3*ty_tests, fa_results)
ax1.set(xlabel='Yield strength [kPa]',
       ylabel='Frontal ablation rate')
ax2.plot(mb_tests, fa_results_mb)
ax2.set(xlabel=r'Terminus mass balance [m/a]')
ax3.plot(v_scalings, fa_results_v)
ax3.set(xlabel=r'Velocity scaling [factor]')

What if we messed up dLdt versus frontal ablation?  Test the interchange.

In [ ]:
dLdt_results_v = [-1*(fa_from_velocity(profile=input_profile, model_velocity=v*np.asarray(input_velocity), terminus_mb=0, verbose=False,
                    tau0=150e3, yield_type='constant', mu=0.01,trim_profile=1) - v*np.asarray(input_velocity)[-1]) for v in v_scalings]

In [ ]:
terminal_velocities = [v*np.asarray(input_velocity)[-1] for v in v_scalings]

fig,(ax1,ax2) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1.plot(v_scalings, dLdt_results_v)
ax1.set(xlabel=r'Velocity scaling [factor]',
       ylabel='dLdt [m/a]' )
ax2.plot(v_scalings, terminal_velocities)
ax2.set(xlabel=r'Velocity scaling [factor]',
        ylabel='Velocity at terminus')

In [ ]:
dLdt_results_v

In [ ]:
fig, ax = plt.subplots()
ax.plot(input_velocity)